# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,21.08,39,27,2.79,CA,1690477814
1,1,springdale,36.1867,-94.1288,34.28,54,0,7.20,US,1690477814
2,2,blackmans bay,-43.0167,147.3167,10.96,77,87,2.36,AU,1690477814
3,3,holualoa,19.6228,-155.9522,25.48,79,0,2.57,US,1690477814
4,4,hadibu,12.6500,54.0333,25.51,76,97,9.31,YE,1690477815


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM")

# Display the map
map1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

city_data_df = city_data_df.loc[(city_data_df["Max Temp"]<27) & 
                                (city_data_df["Max Temp"]>21)&
                                (city_data_df["Wind Speed"]<4.5) &
                                (city_data_df["Cloudiness"]== 0)]


# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,holualoa,19.6228,-155.9522,25.48,79,0,2.57,US,1690477814
56,56,altamont,42.2068,-121.7372,22.44,47,0,2.06,US,1690477830
114,114,maintirano,-18.0667,44.0167,23.25,77,0,4.30,MG,1690477844
177,177,antonio enes,-16.2325,39.9086,21.70,73,0,4.16,MZ,1690477856
214,214,nkhotakota,-12.9274,34.2961,21.21,58,0,1.00,MW,1690477862
221,221,coeur d'alene,47.6777,-116.7805,24.34,37,0,3.09,US,1690477607
361,361,fayzabad,37.1166,70.5800,25.06,22,0,0.47,AF,1690477910
520,520,muynoq,43.7683,59.0214,26.60,51,0,3.92,UZ,1690477947


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

C:\Users\Tommy Vo\AppData\Local\Temp\ipykernel_10468\2866456567.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
3,holualoa,US,19.6228,-155.9522,79,
56,altamont,US,42.2068,-121.7372,47,
114,maintirano,MG,-18.0667,44.0167,77,
177,antonio enes,MZ,-16.2325,39.9086,73,
214,nkhotakota,MW,-12.9274,34.2961,58,
221,coeur d'alene,US,47.6777,-116.7805,37,
361,fayzabad,AF,37.1166,70.5800,22,
520,muynoq,UZ,43.7683,59.0214,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel","apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon =row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params).json()
    
    # Convert the API response to JSON format

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
holualoa - nearest hotel: Kona Hotel
altamont - nearest hotel: Quality Inn
maintirano - nearest hotel: Hotel Safari
antonio enes - nearest hotel: Hotel Quirimbas
nkhotakota - nearest hotel: Zayi Guest House
coeur d'alene - nearest hotel: Resort City Inn
fayzabad - nearest hotel: Kokcha Hotel
muynoq - nearest hotel: Lala


,City,Country,Lat,Lng,Humidity,Hotel Name
3,holualoa,US,19.6228,-155.9522,79,Kona Hotel
56,altamont,US,42.2068,-121.7372,47,Quality Inn
114,maintirano,MG,-18.0667,44.0167,77,Hotel Safari
177,antonio enes,MZ,-16.2325,39.9086,73,Hotel Quirimbas
214,nkhotakota,MW,-12.9274,34.2961,58,Zayi Guest House
221,coeur d'alene,US,47.6777,-116.7805,37,Resort City Inn
361,fayzabad,AF,37.1166,70.5800,22,Kokcha Hotel
520,muynoq,UZ,43.7683,59.0214,51,Lala


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",hover_cols=["Hotel Name","Country"])

# Display the map
map2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)